## Resting state EEG-derived transfer entropy

Code created by Jules Mitchell January 2024.

You are free to use this or any other code from this repository for your own projects and publications. Citation or reference to the repository is not required, but would be much appreciated (see more on README.md).

# Set-Up

In [ ]:
#Import packages
import os
import networkx as nx
import netrd as net
import igraph as ig
from idtxl import idtxl_io as io
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from pathlib import Path

# Define functions
# Mock Data
def generate_adjacency_matrix(Size, N_cells): # Generate a single NxN adjacency matrix with N random integer values
    matrix = np.zeros((Size, Size))
    indices = np.random.choice(Size*Size, N_cells, replace=False)
    matrix.flat[indices] = np.random.randint(1, 9, N_cells)
    return matrix

def generate_directed_graph(N_edges): # generate a single directed graph with N weighted edges
    G = nx.DiGraph() 
    edges_to_add = np.random.choice(range(6), size=(N_edges, 2), replace=False)
    for edge in edges_to_add:
        i, j = edge
        weight = np.random.uniform(0.1, 0.9)
        G.add_edge(i, j, weight=weight)
    return G

def generate_mock_graphs(option, num_graphs, nodes, prob): # Generate mock graphs with various models
    for i in range(num_graphs):
        if option == 1:
            # Erdos-Renyi graph
            graph = nx.erdos_renyi_graph(nodes, prob)
        elif option == 2:
            # Barabasi-Albert graph
            graph = nx.barabasi_albert_graph(nodes,2)
        elif option == 3:
            # Watts-Strogatz graph
            graph = nx.watts_strogatz_graph(nodes, 2, prob)
        elif option == 4:
            # Directed Erdos-Renyi graph
            graph = nx.fast_gnp_random_graph(nodes, prob, directed=True)
        else:
            # Default: Erdos-Renyi graph
            graph = nx.erdos_renyi_graph(nodes, 0.2)
    return graph

# Graph Manipulation 
def combine_graphs(all_graphs, normalise=False):
    average_graph = nx.DiGraph()
    
    N = len(all_graphs) if normalise else 1

    for graph in all_graphs: # may need to enumerate?
        for edge in graph.edges:
            source, target = edge
            if average_graph.has_edge(source, target):
                average_graph[source][target]['weight'] += graph[source][target]['weight']
            else:
                # If the edge doesn't exist in the average_graph, add it
                average_graph.add_edge(source, target, weight=graph[source][target]['weight'])
    
    # Normalize the edge weights after the loop
    for edge in average_graph.edges:
        source, target = edge
        average_graph[source][target]['weight'] /= N

    return average_graph

# Plotting
def graph_plot(graph, with_labels=False, with_weights=False):
    # Settings
    pos = nx.spring_layout(graph)
    with_labels = True if with_labels else False
    
    if not with_weights:
        edge_weights = 1.0
    else:
        edge_weights = [graph[edge[0]][edge[1]].get('weight', 1.0) for edge in graph.edges]

    # Drawing the graph with varying line thickness based on edge weights
    nx.draw_networkx(
        graph, 
        pos, 
        with_labels=with_labels, 
        node_size=700, 
        node_color='skyblue', 
        font_size=8, 
        font_color='black', 
        font_weight='bold', 
        edge_color='gray', 
        width=edge_weights,  # Set the edge thickness based on edge weights
        edge_cmap=plt.cm.Blues  # You can choose a colormap for the edges
    )

    plt.title('Graph Plot')
    plt.show()

# Load IDTxL results
def load_graphs(folder, include = None, weights = 'binary'):
    """
    Load graph data from pickle files in specified folders.

    Parameters:
    - folder (str or Path): Path to the root folder containing subject and session folders.
    - responders (list): List of subjects classified as responder.

    Returns:
    - all_results (list): List of dictionaries containing loaded graph data.
    """
    folder = Path(folder)
    all_results = []

    # Loop through subject folders
    for subject_folder in folder.iterdir():
        if not subject_folder.is_dir():
            continue

        subject_name = subject_folder.name
        if include is not None and subject_name not in include:
            continue
        
        # Loop through session folders
        for session_folder in subject_folder.iterdir():
            if not session_folder.is_dir():
                continue
            
            # Assuming graph files are stored as pickle files
            results = list(session_folder.glob('*.p'))
            
            # Skip processing if there are no pickle files
            if not results:
                print(f"No .p files found in {session_folder}")
                continue

            # Load each graph file into a directed graph
            for result_path in results:
                try:
                    with open(result_path, 'rb') as f:
                        idtxl_file = pickle.load(f)
                except Exception as e:
                    print(f"Error loading {result_path}: {e}")
                    continue

                # Convert to networkx graph
                weights = weights
                adj_matrix = idtxl_file.get_adjacency_matrix(weights=weights, fdr=False) #weight type 
                networkx = io.export_networkx_graph(adjacency_matrix=adj_matrix, weights=weights)
                igraph = Graph.from_networkx(networkx)

                # Determine condition type
                if 'EC' in result_path.stem:
                    task = 'EC'
                else:
                    task = 'EO'

                # Add the information to the list
                all_results.append({
                    "subject": subject_folder.name,
                    'timepoint': session_folder.name,
                    "condition": task,
                    "results": idtxl_file,
                    "networkx": networkx,
                    "igraph": igraph,
                    "adj_matrix": adj_matrix,
                    'weights': weights
                })

    return all_results

def load_classifier_file(file_path):
    # Initialize an empty dictionary to store the loaded data
    loaded_mapping = {}

    # Open the file in read mode ('r')
    with open(file_path, 'r') as file:
        # Read each line in the file
        for line in file:
            # Split each line by the first occurrence of ':' to separate key and value
            key, value = line.split(':', 1)
            # Remove any leading/trailing whitespaces and convert value to list
            loaded_mapping[key.strip()] = eval(value.strip())

    return loaded_mapping

def load_electrodes_file(file_path):
    # Initialize an empty dictionary to store the loaded data
    electrodes = []

    # Open the file in read mode ('r')
    with open(file_path, 'r') as file:
        # Read each line in the file
        for line in file:
            # Strip any extra whitespace (like newline characters) and add to the list
            electrodes.append(line.strip())

    return electrodes
    
def add_classifier(bids, classifier, designation=[1, 0]):

    for entry in bids:
        participant = entry['subject']
        timepoint = entry['timepoint']
        
        found_designation = designation[1]  # Default designation if not found
        
        # Check if the participant is in the response_status_mapping for the given timepoint
        if timepoint in classifier:
            if participant in classifier[timepoint]:
                found_designation = designation[0]
        
        # Assign the found designation to the bid
        entry['group'] = found_designation

def calc_global_efficiency(G):
    n = len(G.nodes())  # Number of nodes in the graph
    total_efficiency = 0
    
    # Iterate over all pairs of nodes
    for i in G.nodes():
        lengths = nx.single_source_shortest_path_length(G, i)  # Shortest path lengths from node i
        for j in G.nodes():
            if i != j:
                if j in lengths:  # Ensure there's a path from i to j
                    total_efficiency += 1 / lengths[j]
                else:
                    total_efficiency += 0  # No path between i and j, contribute 0 efficiency
    
    # Normalize by n(n-1)
    return total_efficiency / (n * (n - 1))

# Global Network Measures
def calculate_global_network_measures(all_graphs):
    # Create an empty list to store data
    data = []

    # Loop through each loaded graph
    for graph_info in all_graphs:
        #igraph = graph_info["igraph"]
        network = graph_info["networkx"]

        # Global measures
        #avg_shrt = igraph.average_path_length()
        global_efficiency = calc_global_efficiency(network) # How efficiently information is exchanged (higher values indicate higher integration) also high segregation?
        
        # Append data to the list
        data.append({
            "subject": graph_info["subject"],
            "group": graph_info["group"],
            "timepoint": graph_info["timepoint"],
            "condition": graph_info["condition"],
            "g_ef": global_efficiency
            })
   
    # Create a DataFrame from the list of data
    global_df = pd.DataFrame(data)

    return global_df

# Local Network Measures
def calculate_local_network_measures(all_graphs, electrode_names):
    # Create an empty list to store data
    clustering_temp = []
    betweenness_temp = []
    indegree_temp = []
    outdegree_temp = []

    # Loop through each loaded graph
    for graph_info in all_graphs:
        subject = graph_info["subject"]
        response_category = graph_info["group"]
        timepoint = graph_info["timepoint"]
        task = graph_info["condition"]
        network = graph_info["networkx"]

        # Local measures
        clustering = nx.clustering(network)
        betweenness = nx.betweenness_centrality(network)
        indegree = network.in_degree() 
        outdegree = network.out_degree()
        
        # Rename columns with electrode names (replace this with your electrode names)
        clustering_renamed = {electrode_names[i]: clustering.get(i, 0) for i in range(len(electrode_names))}
        betweenness_renamed = {electrode_names[i]: betweenness.get(i, 0) for i in range(len(electrode_names))}
        indegree_renamed = {electrode_names[i]: indegree[i] for i in range(len(electrode_names))}
        outdegree_renamed = {electrode_names[i]: outdegree[i] for i in range(len(electrode_names))}

        # Append data to the lists
        clustering_temp.append({
            "subject": subject,
            "group": response_category,
            "timepoint": timepoint,
            "condition": task,
            "measure": "ClCoef",
            ** clustering_renamed
            }) 
         
        betweenness_temp.append({
            "subject": subject,
            "group": response_category,
            "timepoint": timepoint,
            "condition": task,
            "measure": "Btwn",
            ** betweenness_renamed
            })  
        
        indegree_temp.append({
            "subject": subject,
            "group": response_category,
            "timepoint": timepoint,
            "condition": task,
            "measure": "InDgr",
            ** indegree_renamed
            })  
        
        outdegree_temp.append({
            "subject": subject,
            "group": response_category,
            "timepoint": timepoint,
            "condition": task,
            "measure": "OutDgr",
            ** outdegree_renamed
            })  
        
    # Create a DataFrame from the list of data
    clustering_df = pd.DataFrame(clustering_temp)
    betweenness_df = pd.DataFrame(betweenness_temp)
    indegree_df = pd.DataFrame(indegree_temp)
    outdegree_df = pd.DataFrame(outdegree_temp)

    # Combine dataframes
    local_df = pd.concat([clustering_df, betweenness_df, indegree_df, outdegree_df], axis=0)

    # Sort the DataFrame by the index
    local_df.sort_index(inplace=True)

    return local_df 

def te_extraction (all_graphs):

    # Initialize temporary variables
    subject_values = []
    task_values = []
    timepoint_values = []
    response_values = []

    fp_asymmetry_left_values = []
    fp_asymmetry_right_values = []
    ft_asymmetry_left_values = []
    ft_asymmetry_right_values = []
    fo_asymmetry_left_values = []
    fo_asymmetry_right_values = []

    tp_asymmetry_left_values = []
    tp_asymmetry_right_values = []
    to_asymmetry_left_values = []
    to_asymmetry_right_values = []

    po_asymmetry_left_values = []
    po_asymmetry_right_values = []

    # Iterate through each participant-file in the list
    for participant in range(len(all_graphs)):

        # Load participant details
        subject = all_graphs[participant]['subject']
        task = all_graphs[participant]['condition']
        timepoint = all_graphs[participant]['timepoint']
        response_category = all_graphs[participant]['group']

        # Load the channel weights list per participant-file
        channel_weights_list = all_graphs[participant]['adj_matrix'].get_edge_list() # where the weights are the te values

        # Initialize sums
        fp_caudal_left_sum = 0
        fp_caudal_right_sum = 0
        fp_rostral_left_sum = 0
        fp_rostral_right_sum = 0

        # Iterate through the list and apply the conditional statement
        for i, j, weight in channel_weights_list:
            
            # Left hemisphere frontal to parietal: Fp1, AF3, F7, F3, Fz --> FC1, C3, CP1, P3, Pz, Cz
            if (i in [0, 1, 2, 3, 30]) and (j in [4, 7, 8, 11, 12, 31]):
                fp_caudal_left_sum += weight

            # Right hemisphere frontal to parietal: Fp2, AF4, F8, F4, Fz, Pz --> FC2, C4, CP2, P4, Pz, Cz
            if (i in [29, 28, 27, 26, 30]) and (j in [25, 22, 21, 18, 12, 31]):
                fp_caudal_right_sum += weight

            # Left hemisphere parietal to frontal: FC1, C3, CP1, P3, Pz, Cz --> Fp1, AF3, F7, F3, Fz
            if (i in [4, 7, 8, 11, 12, 31]) and (j in [0, 1, 2, 3, 30]):
                fp_rostral_left_sum += weight

            # Right hemisphere parietal to frontal: FC2, C4, CP2, P4, Pz, Cz --> Fp2, AF4, F8, F4, Fz
            if (i in [25, 22, 21, 18, 12, 31]) and (j in [29, 28, 27, 26, 30]):
                fp_rostral_right_sum += weight

        # Calculate the final values by scaling by the 1/number of node combinations
        fp_caudal_left_final_value = fp_caudal_left_sum * (1/30)
        fp_caudal_right_final_value = fp_caudal_right_sum * (1/30)
        fp_rostral_left_final_value = fp_rostral_left_sum * (1/30)
        fp_rostral_right_final_value = fp_rostral_right_sum * (1/30)

        # Calculate asymmetry values for frontal-parietal
        if (fp_caudal_left_final_value + fp_rostral_left_final_value) == 0:
            fp_asymmetry_left = 0.5
        else:
            fp_asymmetry_left = (fp_caudal_left_final_value - fp_rostral_left_final_value) / (fp_caudal_left_final_value + fp_rostral_left_final_value)

            # if you want to transform to 0-1 scale
            # fp_asymmetry_left_transformed = (fp_asymmetry_left + 1) / 2
            # fp_asymmetry_left_transformed = round(fp_asymmetry_left_transformed, 3)  # Round to 3 decimal places

        if (fp_caudal_right_final_value + fp_rostral_right_final_value) == 0:
            fp_asymmetry_right = 0.5
        else:
            fp_asymmetry_right = (fp_caudal_right_final_value - fp_rostral_right_final_value) / (fp_caudal_right_final_value + fp_rostral_right_final_value)
            # fp_asymmetry_right_transformed = (fp_asymmetry_right + 1) / 2
            # fp_asymmetry_right_transformed = round(fp_asymmetry_right_transformed, 3)  # Round to 3 decimal places
        
        # Append asymmetry values as needed
        fp_asymmetry_left_values.append(fp_asymmetry_left)
        fp_asymmetry_right_values.append(fp_asymmetry_right)

        # Initialize sums
        ft_caudal_left_sum = 0
        ft_caudal_right_sum = 0
        ft_rostral_left_sum = 0
        ft_rostral_right_sum = 0

        # Iterate through the list and apply the conditional statement
        for i, j, weight in channel_weights_list:
            
            # Left hemisphere frontal to temporal: Fp1, AF3, F7, F3, Fz --> FC5, T7, CP5, P7
            if (i in [0, 1, 2, 3, 30]) and (j in [5, 6, 9, 10]):
                ft_caudal_left_sum += weight

            # Right hemisphere frontal to temporal: Fp2, AF4, F8, F4, Fz --> P8, CP6, T8, FC6
            if (i in [29, 28, 27, 26, 30]) and (j in [24, 23, 20, 19]):
                ft_caudal_right_sum += weight

            # Left hemisphere temporal to frontal: FC5, T7, CP5, P7 --> Fp1, AF3, F7, F3, Fz
            if (i in [5, 6, 9, 10]) and (j in [0, 1, 2, 3, 30]):
                ft_rostral_left_sum += weight

            # Right hemisphere temporal to frontal: P8, CP6, T8, FC6 --> Fp2, AF4, F8, F4, Fz
            if (i in [24, 23, 20, 19]) and (j in [29, 28, 27, 26, 30]):
                ft_rostral_right_sum += weight

        # Calculate the final values by scaling by the 1/number of pairs
        ft_caudal_left_final_value = ft_caudal_left_sum * (1/20)
        ft_caudal_right_final_value = ft_caudal_right_sum * (1/20)
        ft_rostral_left_final_value = ft_rostral_left_sum * (1/20)
        ft_rostral_right_final_value = ft_rostral_right_sum * (1/20)

        # Calculate asymmetry values for fronto-temporal
        if (ft_caudal_left_final_value + ft_rostral_left_final_value) == 0:
            ft_asymmetry_left = 0.5
        else:
            ft_asymmetry_left = (ft_caudal_left_final_value - ft_rostral_left_final_value) / (ft_caudal_left_final_value + ft_rostral_left_final_value)

            # if you want to transform to 0-1 scale
            # ft_asymmetry_left_transformed = (ft_asymmetry_left + 1) / 2
            # ft_asymmetry_left_transformed = round(ft_asymmetry_left_transformed, 3)  # Round to 3 decimal places

        if (ft_caudal_right_final_value + ft_rostral_right_final_value) == 0:
            ft_asymmetry_right = 0.5
        else:
            ft_asymmetry_right = (ft_caudal_right_final_value - ft_rostral_right_final_value) / (ft_caudal_right_final_value + ft_rostral_right_final_value)

            # if you want to transform to 0-1 scale
            # ft_asymmetry_right_transformed = (ft_asymmetry_right + 1) / 2
            # ft_asymmetry_right_transformed = round(ft_asymmetry_right_transformed, 3)  # Round to 3 decimal places

        # Append asymmetry values as needed
        ft_asymmetry_left_values.append(ft_asymmetry_left)
        ft_asymmetry_right_values.append(ft_asymmetry_right)

        # Initialize sums
        fo_caudal_left_sum = 0
        fo_caudal_right_sum = 0
        fo_rostral_left_sum = 0
        fo_rostral_right_sum = 0

        # Iterate through the list and apply the conditional statement
        for i, j, weight in channel_weights_list:
            
            # Left hemisphere frontal to occipital: Fp1, AF3, F7, F3, Fz --> PO3, O1, Oz
            if (i in [0, 1, 2, 3, 30]) and (j in [13, 14, 15]):
                fo_caudal_left_sum += weight

            # Right hemisphere frontal to occipital: Fp2, AF4, F8, F4, Fz --> PO4, O2, Oz
            if (i in [29, 28, 27, 26, 30]) and (j in [17, 16, 15]):
                fo_caudal_right_sum += weight

            # Left hemisphere occipital to frontal: PO3, O1, Oz --> Fp1, AF3, F7, F3, Fz
            if (i in [13, 14, 15]) and (j in [0, 1, 2, 3, 30]):
                fo_rostral_left_sum += weight

            # Right hemisphere occipital to frontal: PO4, O2, Oz --> Fp2, AF4, F8, F4, Fz
            if (i in [17, 16, 15]) and (j in [29, 28, 27, 26, 30]):
                fo_rostral_right_sum += weight

        # Calculate the final values by scaling by the 1/number of pairs
        fo_caudal_left_final_value = fo_caudal_left_sum * (1/15)
        fo_caudal_right_final_value = fo_caudal_right_sum * (1/15)
        fo_rostral_left_final_value = fo_rostral_left_sum * (1/15)
        fo_rostral_right_final_value = fo_rostral_right_sum * (1/15)

        # Calculate asymmetry values for fronto-occipital
        if (fo_caudal_left_final_value + fo_rostral_left_final_value) == 0:
            fo_asymmetry_left = 0.5
        else:
            fo_asymmetry_left = (fo_caudal_left_final_value - fo_rostral_left_final_value) / (fo_caudal_left_final_value + fo_rostral_left_final_value)

            # if you want to transform to 0-1 scale
            # fo_asymmetry_left_transformed = (fo_asymmetry_left + 1) / 2
            # fo_asymmetry_left_transformed = round(fo_asymmetry_left_transformed, 3)  # Round to 3 decimal places

        if (fo_caudal_right_final_value + fo_rostral_right_final_value) == 0:
            fo_asymmetry_right = 0.5
        else:
            fo_asymmetry_right = (fo_caudal_right_final_value - fo_rostral_right_final_value) / (fo_caudal_right_final_value + fo_rostral_right_final_value)

            # if you want to transform to 0-1 scale
            # fo_asymmetry_right_transformed = (fo_asymmetry_right + 1) / 2
            # fo_asymmetry_right_transformed = round(fo_asymmetry_right_transformed, 3)  # Round to 3 decimal places

        # Append asymmetry values as needed
        fo_asymmetry_left_values.append(fo_asymmetry_left)
        fo_asymmetry_right_values.append(fo_asymmetry_right)

        # Initialize sums
        tp_caudal_left_sum = 0
        tp_caudal_right_sum = 0
        tp_rostral_left_sum = 0
        tp_rostral_right_sum = 0

        # Iterate through the list and apply the conditional statement
        for i, j, weight in channel_weights_list:
            
            # Left hemisphere temporal to parietal: FC5, T7, CP5, P7 --> FC1, C3, CP1, P3, Pz, Cz
            if (i in [5, 6, 9, 10]) and (j in [4, 7, 8, 11, 12, 31]):
                tp_caudal_left_sum += weight

            # Right hemisphere temporal to parietal: P8, CP6, T8, FC6 --> FC2, C4, CP2, P4, Pz, Cz
            if (i in [24, 23, 20, 19]) and (j in [25, 22, 21, 18, 12, 31]):
                tp_caudal_right_sum += weight

            # Left hemisphere parietal to temporal: FC1, C3, CP1, P3, Pz, Cz --> FC5, T7, CP5, P7
            if (i in [4, 7, 8, 11, 12, 31]) and (j in [5, 6, 9, 10]):
                tp_rostral_left_sum += weight

            # Right hemisphere parietal to temporal: FC2, C4, CP2, P4, Pz, Cz --> P8, CP6, T8, FC6
            if (i in [25, 22, 21, 18, 12, 31]) and (j in [24, 23, 20, 19]):
                tp_rostral_right_sum += weight

        # Calculate the final values by scaling by the 1/number of pairs
        tp_caudal_left_final_value = tp_caudal_left_sum * (1/24)
        tp_caudal_right_final_value = tp_caudal_right_sum * (1/24)
        tp_rostral_left_final_value = tp_rostral_left_sum * (1/24)
        tp_rostral_right_final_value = tp_rostral_right_sum * (1/24)

        # Calculate asymmetry values for temporal-parietal
        if (tp_caudal_left_final_value + tp_rostral_left_final_value) == 0:
            tp_asymmetry_left = 0.5
        else:
            tp_asymmetry_left = (tp_caudal_left_final_value - tp_rostral_left_final_value) / (tp_caudal_left_final_value + tp_rostral_left_final_value)

            # if you want to transform to 0-1 scale
            # tp_asymmetry_left_transformed = (tp_asymmetry_left + 1) / 2
            # tp_asymmetry_left_transformed = round(tp_asymmetry_left_transformed, 3)  # Round to 3 decimal places

        if (tp_caudal_right_final_value + tp_rostral_right_final_value) == 0:
            tp_asymmetry_right = 0.5
        else:
            tp_asymmetry_right = (tp_caudal_right_final_value - tp_rostral_right_final_value) / (tp_caudal_right_final_value + tp_rostral_right_final_value)

            # if you want to transform to 0-1 scale
            # tp_asymmetry_right_transformed = (tp_asymmetry_right + 1) / 2
            # tp_asymmetry_right_transformed = round(tp_asymmetry_right_transformed, 3)  # Round to 3 decimal places

        # Print or return the asymmetry values as needed
        tp_asymmetry_left_values.append(tp_asymmetry_left)
        tp_asymmetry_right_values.append(tp_asymmetry_right)

        # Initialize sums
        to_caudal_left_sum = 0
        to_caudal_right_sum = 0
        to_rostral_left_sum = 0
        to_rostral_right_sum = 0

        # Iterate through the list and apply the conditional statement
        for i, j, weight in channel_weights_list:
            
            # Left hemisphere temporal to occipital: T7, CP5, P7, FC5 --> PO3, O1, Oz
            if (i in [5, 6, 9, 10]) and (j in [13, 14, 15]):
                to_caudal_left_sum += weight

            # Right hemisphere temporal to occipital: P8, CP6, T8, FC6 --> PO4, O2, Oz
            if (i in [24, 23, 20, 19]) and (j in [17, 16, 15]):
                to_caudal_right_sum += weight

            # Left hemisphere occipital to temporal: PO3, O1, Oz --> T7, CP5, P7, FC5
            if (i in [13, 14, 15]) and (j in [5, 6, 9, 10]):
                to_rostral_left_sum += weight

            # Right hemisphere occipital to temporal: PO4, O2, Oz --> P8, CP6, T8, FC6
            if (i in [17, 16, 15]) and (j in [24, 23, 20, 19]):
                to_rostral_right_sum += weight

        # Calculate the final values by scaling by the 1/number of pairs
        to_caudal_left_final_value = to_caudal_left_sum * (1/12)
        to_caudal_right_final_value = to_caudal_right_sum * (1/12)
        to_rostral_left_final_value = to_rostral_left_sum * (1/12)
        to_rostral_right_final_value = to_rostral_right_sum * (1/12)

        # Calculate asymmetry values for temporal-occipital
        if (to_caudal_left_final_value + to_rostral_left_final_value) == 0:
            to_asymmetry_left = 0.5
        else:
            to_asymmetry_left = (to_caudal_left_final_value - to_rostral_left_final_value) / (to_caudal_left_final_value + to_rostral_left_final_value)

            # if you want to transform to 0-1 scale
            # to_asymmetry_left_transformed = (to_asymmetry_left + 1) / 2
            # to_asymmetry_left_transformed = round(to_asymmetry_left_transformed, 3)  # Round to 3 decimal places

        if (to_caudal_right_final_value + to_rostral_right_final_value) == 0:
            to_asymmetry_right = 0.5
        else:
            to_asymmetry_right = (to_caudal_right_final_value - to_rostral_right_final_value) / (to_caudal_right_final_value + to_rostral_right_final_value)

            # if you want to transform to 0-1 scale
            # to_asymmetry_right_transformed = (to_asymmetry_right + 1) / 2
            # to_asymmetry_right_transformed = round(to_asymmetry_right_transformed, 3)  # Round to 3 decimal places

        # Print or return the asymmetry values as needed
        to_asymmetry_left_values.append(to_asymmetry_left)
        to_asymmetry_right_values.append(to_asymmetry_right)

        # Initialize sums
        po_caudal_left_sum = 0
        po_caudal_right_sum = 0
        po_rostral_left_sum = 0
        po_rostral_right_sum = 0

        # Iterate through the list and apply the conditional statement
        for i, j, weight in channel_weights_list:
            
            # Left hemisphere parietal to occipital: FC1, C3, CP1, P3, Pz, Cz --> PO3, O1, Oz
            if (i in [4, 7, 8, 11, 12, 31]) and (j in [13, 14, 15]):
                po_caudal_left_sum += weight

            # Right hemisphere parietal to occipital: FC2, C4, CP2, P4, Pz, Cz --> PO4, O2, Oz
            if (i in [25, 22, 21, 18, 12, 31]) and (j in [17, 16, 15]):
                po_caudal_right_sum += weight

            # Left hemisphere occipital to parietal: PO3, O1, Oz --> FC1, C3, CP1, P3, Pz, Cz
            if (i in [13, 14, 15]) and (j in [4, 7, 8, 11, 12, 31]):
                po_rostral_left_sum += weight

            # Right hemisphere occipital to parietal: PO4, O2, Oz --> FC2, C4, CP2, P4, Pz, Cz
            if (i in [17, 16, 15]) and (j in [25, 22, 21, 18, 12, 31]):
                po_rostral_right_sum += weight

        # Calculate the final values by scaling by the 1/number of pairs
        po_caudal_left_final_value = po_caudal_left_sum * (1/18)
        po_caudal_right_final_value = po_caudal_right_sum * (1/18)
        po_rostral_left_final_value = po_rostral_left_sum * (1/18)
        po_rostral_right_final_value = po_rostral_right_sum * (1/18)

        # Calculate asymmetry values for parieto-occipital
        if (po_caudal_left_final_value + po_rostral_left_final_value) == 0:
            po_asymmetry_left = 0.5
        else:
            po_asymmetry_left = (po_caudal_left_final_value - po_rostral_left_final_value) / (po_caudal_left_final_value + po_rostral_left_final_value)

            # if you want to transform to 0-1 scale
            # po_asymmetry_left_transformed = (po_asymmetry_left + 1) / 2
            # po_asymmetry_left_transformed = round(po_asymmetry_left_transformed, 3)  # Round to 3 decimal places

        if (po_caudal_right_final_value + po_rostral_right_final_value) == 0:
            po_asymmetry_right = 0.5
        else:
            po_asymmetry_right = (po_caudal_right_final_value - po_rostral_right_final_value) / (po_caudal_right_final_value + po_rostral_right_final_value)

            # if you want to transform to 0-1 scale
            # po_asymmetry_right_transformed = (po_asymmetry_right + 1) / 2
            # po_asymmetry_right_transformed = round(po_asymmetry_right_transformed, 3)  # Round to 3 decimal places

        # Print or return the asymmetry values as needed
        po_asymmetry_left_values.append(po_asymmetry_left)
        po_asymmetry_right_values.append(po_asymmetry_right)

        # # Append identifiers
        subject_values.append(subject)
        task_values.append(task)
        timepoint_values.append(timepoint)
        response_values.append(response_category)

        # Save the results to a DataFrame with an ID column
        te_extracted_df = pd.DataFrame({
            'subject': subject_values,
            'group': response_values,
            'condition': task_values,
            'timepoint': timepoint_values, # add subject ID, task, timepoint
            'FP_Asymmetry_Left': fp_asymmetry_left_values,
            'FP_Asymmetry_Right': fp_asymmetry_right_values,
            'FT_Asymmetry_Left': ft_asymmetry_left_values,
            'FT_Asymmetry_Right': ft_asymmetry_right_values,            
            'FO_Asymmetry_Left': fo_asymmetry_left_values,
            'FO_Asymmetry_Right': fo_asymmetry_right_values,
            'TP_Asymmetry_Left': tp_asymmetry_left_values,
            'TP_Asymmetry_Right': tp_asymmetry_right_values,
            'TO_Asymmetry_Left': to_asymmetry_left_values,
            'TO_Asymmetry_Right': to_asymmetry_right_values,
            'PO_Asymmetry_Left': po_asymmetry_left_values,
            'PO_Asymmetry_Right': po_asymmetry_right_values
            })

    return te_extracted_df  

def questionnaire_combine(core_data, questionnare_data, processing_data):

	# Rename columns and select a sub-set
	questionnare_data.rename(columns={"base_id": "Subject", 'age': 'Age', 'gender_cat':"Sex", "BSS_score.00A": "BSS_ses01", "BSS_score.06D": "BSS_ses02", 
									  "BSS_score.07A":"BSS_ses03", "MADRS_score.00A": "MADRS_ses01", "MADRS_score.07A": "MADRS_ses03", 'DASS_Dep.00A': 'DASSdep_ses01', 
                                      'DASS_Dep.06D': 'DASSdep_ses02', 'DASS_Dep.07A': 'DASSdep_ses03', 'DASS_Anx.00A': 'DASSanx_ses01', 
                                      'DASS_Anx.06D': 'DASSanx_ses02', 'Dass21_anxiety.07A': 'DASSanx_ses03', 'DASS_Stress.00A': 'DASSstress_ses01', 
                                      'DASS_Stress.06D': 'DASSstress_ses02', 'Dass21_stress.07A': 'DASSstress_ses03', 
									  "BSS_Responder_cat": "Response_Post", "BSS_Responder_FUP_cat": "Response_FUP"}, inplace = True)
	
	questionnare_data = questionnare_data[['Subject', 'Sex', 'Age','BSS_ses01', 'BSS_ses02', 'BSS_ses03', 'MADRS_ses01', 'MADRS_ses03', 
                                        'DASSdep_ses01', 'DASSdep_ses02', 'DASSdep_ses03','DASSanx_ses01', 'DASSanx_ses02', 'DASSanx_ses03',
                                        'DASSstress_ses01', 'DASSstress_ses02', 'DASSstress_ses03','Response_Post', 'Response_FUP']]
	
	# Align subject ID's for combining.
	questionnare_data['Subject'].replace('OKTOS_00', 'sub-', regex=True, inplace=True)

	# Loop through each subject and BSS timepoint value in qualtrics df and add to the corresponding subject/Tx in the master
	for s, bss_value in zip(questionnare_data['Subject'], questionnare_data['BSS_ses01']):
		condition = (core_data['subject'] == s) & (core_data['timepoint'] == 'ses-01')
		core_data.loc[condition, 'BSS'] = bss_value

	for s, bss_value in zip(questionnare_data['Subject'], questionnare_data['BSS_ses02']):
		condition = (core_data['subject'] == s) & (core_data['timepoint'] == 'ses-02')
		core_data.loc[condition, 'BSS'] = bss_value

	for s, bss_value in zip(questionnare_data['Subject'], questionnare_data['BSS_ses03']):
		condition = (core_data['subject'] == s) & (core_data['timepoint'] == 'ses-03')
		core_data.loc[condition, 'BSS'] = bss_value

	for s, madrs_value in zip(questionnare_data['Subject'], questionnare_data['MADRS_ses01']):
		condition = (core_data['subject'] == s) & (core_data['timepoint'] == 'ses-01')
		core_data.loc[condition, 'MADRS'] = madrs_value

	for s, madrs_value in zip(questionnare_data['Subject'], questionnare_data['MADRS_ses03']):
		condition = (core_data['subject'] == s) & (core_data['timepoint'] == 'ses-03')
		core_data.loc[condition, 'MADRS'] = madrs_value

	for s, sex in zip(questionnare_data['Subject'], questionnare_data['Sex']):
		condition = (core_data['subject'] == s)
		core_data.loc[condition, 'Sex'] = sex

	for s, age in zip(questionnare_data['Subject'], questionnare_data['Age']):
		condition = (core_data['subject'] == s)
		core_data.loc[condition, 'Age'] = age

	for s, time in zip(processing_data['subj_ID'], processing_data['tx']):
		condition = (core_data['subject'] == s) & (core_data['timepoint'] == time)
		core_data.loc[condition, 'is_before_12'] = processing_data['is_before_12']

	core_data['MADRS'].replace(0, 999,inplace=True)

	return core_data

In [ ]:
# Set directory and file paths using pathlib
root_dir = Path("C:/Users/j_m289/Pictures/phd/3. Data Analysis/studies/OKTOS/resting_te")
data_folder = root_dir / "data/derivatives" 
analysis_folder = root_dir / "analysis"
response_file = root_dir / 'response_status_mapping.txt'
electrode_file = root_dir / 'electrode_names.txt'
questionnaire_data = analysis_folder / 'spreadsheets' / 'OKTOS_Qualtrics_master.csv'
processing_data = analysis_folder / 'spreadsheets' / 'Recording_parameters.csv'

# Load response status classification file
response_mapping = load_classifier_file(response_file)

# Load electrode names (used for converting source-target pairs from IDTXL to channel names)
electrode_names = load_electrodes_file(electrode_file)
electrode_dict = {i: electrode for i, electrode in enumerate(electrode_names)}

# Load relevant data to combine with extracted metrics
qualtrics = pd.read_csv(questionnaire_data)
eeg_scantime = pd.read_csv(processing_data)

# Post-Transfer Entropy Analyses

In [ ]:
# Load files (Note, there is currently an issue with the gml file load (something to do with strings))
all_graphs = load_graphs(data_folder, weights='binary')

# Add response status info
add_classifier(all_graphs, response_mapping, designation=[1, 0])

### TE Extraction

In [ ]:
# Extract TE values for significant source-target pairs and calculate information flow metrics
te_data = te_extraction(all_graphs)

### Topographical Analysis

In [ ]:
# Global measures
global_data = calculate_global_network_measures(all_graphs)

# Local measures
local_data = calculate_local_network_measures(all_graphs, electrode_names)

### Dataframe re-arrangement

In [ ]:
# Melt the DataFrames to reshape so that regions/channels labels are in one column and the metrics are in another. 
te_data_long = pd.melt(te_data, id_vars=['subject', 'group', 'condition', 'timepoint'], 
                  value_vars=['FP_Asymmetry_Left', 'FP_Asymmetry_Right', 
                              'FT_Asymmetry_Left', 'FT_Asymmetry_Right',
                              'FO_Asymmetry_Left', 'FO_Asymmetry_Right',
                              'TP_Asymmetry_Left', 'TP_Asymmetry_Right',
                              'TO_Asymmetry_Left', 'TO_Asymmetry_Right',
                              'PO_Asymmetry_Left', 'PO_Asymmetry_Right'],
                  var_name='regions', value_name='asymmetry')

# Clean the 'regions' column to remove the 'Asymmetry' and 'Asymmetry_Right' parts
te_data_long['regions'] = te_data_long['regions'].str.replace(r'_Asymmetry', '', regex=True)

# Melt the DataFrame to reshape the columns starting from 'FP_Asymmetry_Left'
local_data_long = pd.melt(local_data, id_vars=['subject', 'group', 'condition', 'timepoint', 'measure'], 
                  value_vars=electrode_names,
                  var_name='channels', value_name='value')


In [ ]:
# Define regions for local_data
regions_dict = {
    'Frontal': {
        'Left': ['Fp1', 'AF3', 'F7', 'F3'],
        'Right': ['Fp2', 'AF4', 'F8', 'F4'],
        'Midline': ['Fz']
    },
    'Central': {
        'Midline': ['Cz']
    },
    'Parietal': {
        'Left': ['FC1', 'C3', 'CP1', 'P3'],
        'Right': ['FC2', 'C4', 'CP2', 'P4'],
        'Midline': ['Pz']
    },
    'Temporal': {
        'Left': ['FC5', 'T7', 'CP5', 'P7'],
        'Right': ['P8', 'CP6', 'T8', 'FC6']
    },
    'Occipital': {
        'Left': ['PO3', 'O1'],
        'Right': ['PO4', 'O2'],
        'Midline': ['Oz']
    }
}

# Function to get the region for a given channel
def get_region(channel):
    for region, sides in regions_dict.items():
        for side, channels in sides.items():
            if channel in channels:
                return f"{region}_{side}"
    return None  # Return None if channel is not found

# Apply the function to create the 'region' column in the local_data_long
local_data_long['region'] = local_data_long['channels'].apply(get_region)

### Data Concatenation and Exporting

In [ ]:
te_data_long = questionnaire_combine(core_data=te_data_long, questionnare_data=qualtrics, processing_data=eeg_scantime)
global_data_long = questionnaire_combine(core_data=global_data, questionnare_data=qualtrics, processing_data=eeg_scantime)
local_data_long = questionnaire_combine(core_data=local_data_long, questionnare_data=qualtrics, processing_data=eeg_scantime)

In [ ]:
te_data_long.to_csv(analysis_folder/'spreadsheets/OKTOS_te_asymmetry.csv', index=False)
global_data_long.to_csv(analysis_folder/'spreadsheets/OKTOS_te_global.csv', index=False)
local_data_long.to_csv(analysis_folder/'spreadsheets/OKTOS_te_local.csv', index=True)